### Imports & Settings

In [1]:
from collections import Counter
import time
import numpy as np
import pandas as pd
import nn_model as nn
from utils import *
from utils import properties as p
import hashlib
import matplotlib.pyplot as plt
import itertools
from sklearn.metrics import f1_score
from metrics import *
from IPython.display import display

In [2]:
from noknow.core import ZK, ZKSignature, ZKParameters, ZKData, ZKProof
import copy

### Parameters

In [3]:
epochs_n = 1
learning_rate = 0.1
seed = 19101995

features = {
    'ETHNICITY': ['Asian', 'Black', 'Indian', 'White'],
    'GENDER': ['Female', 'Male']
}

not_features = {
    'ETHNICITY': ['ROCK', 'PAPER', 'SCISSORS', 'UNCATEGORIZED'],
    'GENDER': ['ZERO', 'ONE']
}

p.update({
    
    #Age range
    'age_min' : 0,
    'age_max' : 39,
    
    #Number of data samples
    'data_samples' : 10000,
    
})

### ZKP Server

In [4]:
class ZKServer:
    def __init__(self, password, features):
        self.password = password
        self.features = features
        self.client_representations = {}
        self.tokens = {}
        self.initialized_zk_features = False
        
        for el in features:
            self.client_representations[el] = []
            self.tokens[el] = []
        self.zk = ZK.new(curve_name="secp384r1", hash_alg="sha3_512")
        self.signature: ZKSignature = self.zk.create_signature(password)
            
        self.groups = {}
        for f in self.features:
            self.groups[f] = Counter()
            
    def load_client_signature(self, signature):
        #load client signature
        client_signature = ZKSignature.load(signature)
        return ZK(client_signature.params)
    
    def create_token(self, feature, signature):
        #load client signature
        client_signature = ZKSignature.load(signature)
        client_zk = ZK(client_signature.params)
        # Create a signed token and send to the client
        token = self.zk.sign(self.password, client_zk.token()).dump(separator=":")
        self.tokens[feature] += [token]
        self.client_representations[feature] += [{'signature' : client_signature, 'zk' : client_zk}]
        assert token in self.tokens[feature]
        return token
        
    def verify_clients(self, feature, signature):
        # Get the token from the client
        proof = ZKData.load(signature)
        token = ZKData.load(proof.data, ":")
        
        proofs = []
        # In this example, the server signs the token so it can be sure it has not been modified
        for label in self.client_representations[feature]:
            client_signature = label['signature']
            client_zk = label['zk']
            proofs += [self.verify_client(proof, token, client_signature, client_zk)]
            
        return sum(proofs) >= 1
    
    def verify_client(self, proof, token, client_signature, client_zk):
        return self.zk.verify(token, self.signature) and \
                client_zk.verify(proof, client_signature, data=token)
    
    def verify(self, feature, signatures):
        return sum([self.verify_clients(feature, s) for s in signatures]) >= 1
    
    def get_features_permutation(self):
        balanced = []
        for f in self.encrypted_features_dict:
            balanced += [self.encrypted_features_dict[f]]
        return list(itertools.product(*balanced))
    
    def init_zk(self, clients):
        if self.initialized_zk_features:
            print('Server already initialized')
            return
        
        feature_dict = {}
        
        for c in clients:
            self.create_token(c.feature, c.signature)
            
            if c.feature not in feature_dict:
                feature_dict[c.feature] = [{'feature': c.feature, 'label': c.encrypt_label()}]
            else:
                feature_dict[c.feature] += [{'feature': c.feature, 'label': c.encrypt_label()}]
        
        self.encrypted_features_dict = feature_dict
        
        permutations = self.get_features_permutation()
                
        for individual in permutations:
            for feat in individual:
                self.groups[feat['feature']].update({feat['label']: 1})
        self.initialized_zk_features = True

### ZKP Client

In [5]:
class ZKClient:
    def __init__(self, feature, label):
        self.zk = ZK.new(curve_name="secp256k1", hash_alg="sha3_256")
        self.label = label
        self.feature = feature
        # Create signature and send to server
        self.signature = self.zk.create_signature(label).dump()
        
    def set_label(self, label):
        self.label = label
        self.signature = self.zk.create_signature(label).dump()
            
    def create_proof(self, server_token):
        # Create a proof that signs the provided token and sends to server
        proof = self.zk.sign(self.label, server_token).dump()
        return proof
    
    def create_proofs(self, server_tokens):
        return [self.create_proof(t) for t in server_tokens]
    
    def encrypt_label(self, encrypt=True):
        #Create a secret hash
        secret = hashlib.sha256()
        #use the zk client salt value to encrypt the string
        encrypted_label = self.label + str(self.zk.params.s)
        #create the hash
        secret.update(encrypted_label.encode('utf-8'))
        
        if encrypt:
            return secret.hexdigest()
        else:
            return self.label #TESTING ONLY

### Server Initializer

In [6]:
class ServerInitializer:
    def __init__(self, features, client_prototype):
        self.client_prototype = client_prototype
        self.features = features
        
    def create_clients(self):
        clients = []
        for feature in self.features:
            for label in self.features[feature]:
                client = copy.copy(self.client_prototype)
                client.feature = feature
                client.set_label(label)
                clients += [client]
        return clients

### Client Initializer

In [7]:
class ClientInitializer:
    def __init__(self):
        self.generated = False
    def generate_client_prototype(self):
        if not self.generated:
            self.generated = True
            return ZKClient('', '')

### Model classes

In [8]:
#Server 
class FairServer(ZKServer):
    def __init__(self, password, features, model, max_gap, gap, self_balancing=True):
        super().__init__(password, features)
        self.self_balancing = self_balancing
        self.model = LearningModel(model)
        self.workers = []
        self.queue = {}
        self.max_gap = max_gap
        self.gap = gap
        
        
    def register_worker(self, worker):
        if self.zk_authenticate(worker):
            self.workers += [worker]
        
    def register_feature_group(self, feature, label):
        self.groups[feature].update({label : 1})

    def is_balanced(self, feature, label):
        if len(self.groups[feature]) == 0:
            return True
        count = sum(count for n, count in self.groups[feature].items())
        avg = count / len(self.groups[feature])
        calc_gap = (self.groups[feature][label] + 1) - avg
        #print(calc_gap/avg, calc_gap)
        return (calc_gap / avg) <= self.gap or calc_gap <= self.max_gap
    
    def send_model(self, worker):
        worker.load_model(self.model)
        
    def balanced_update(self, features):
        #if self balancin is disabled return true
        if not self.self_balancing:
            return True
        
        return len(features) == sum([
            self.is_balanced(list(f.keys())[0], list(f.values())[0])
            for f 
            in features
        ])
    
    def get_features_key_values(self, f):
        return list(f.keys())[0], list(f.values())[0]
        
    def enqueue_worker(self, w_id, features):
        labels = 0
        for f in features: 
            k, v = self.get_features_key_values(f)
            labels += int(v, 16) #hex
        if str(labels) not in self.queue:
            self.queue[str(labels)] = [w_id]
        else:
            self.queue[str(labels)] += [w_id]
    
    def load_update(self, w_id, features, weights):
        if not self.balanced_update(features):
            #print('Unbalanced id: ' + str(w_id))
            self.enqueue_worker(w_id, features)
            return
        for f in features:
            k, v = self.get_features_key_values(f)
            self.register_feature_group(k, v)
        self.model.set_weights(weights)
        
    def request_training(self, w):
        w.train()
        
    def zk_authenticate(self, w):
        #ZKP Authentication
        verification = [
            self.verify_clients(f, w.zkp_clients[f].create_proof(self.tokens[f][0]))
            for f in self.features                                    
        ]
        auth = sum(verification) == len(self.features)
        #print(('' if auth else 'Un') + 'authorized')
        return auth
    
    def get_next_possible_updates(self):
        possible = []
        permutation = self.get_features_permutation()
        for individual in permutation:
            bal = [self.is_balanced(feat['feature'], feat['label']) for feat in individual]
            if False not in bal:
                possible += [individual]
        return possible
    
    def get_possible_queues(self):
        possible_updates = self.get_next_possible_updates()
        queue_ids = []
        for individual in possible_updates:
            queue_ids += [str(sum([int(feat['label'], 16) for feat in individual]))]
        return queue_ids
    
    def get_possible_workers(self):
        q_ids = self.get_possible_queues()
        possible_ids = [
            q_id 
            for q_id in q_ids 
            if q_id in self.queue and len(self.queue[q_id]) != 0
        ]
        return self.queue[possible_ids[0]] if len(possible_ids) != 0 else []
    
    def get_worker(self, w_id):
        return next(filter(lambda w: w.id == w_id, self.workers))
    
    def train(self):
        #Sequential train of workers
        for w in self.workers:
            self.send_model(w)
            self.request_training(w)
        
        #Balance the rest of the workers
        possible_workers = self.get_possible_workers()
        while len(possible_workers) != 0:
            w_id = possible_workers.pop()
            w = self.get_worker(w_id)
            self.send_model(w)
            self.request_training(w)
            possible_workers = self.get_possible_workers()
        
    def fair_metrics(self):
        return 'Fair metrics: '
    
    def predict(self, x):
        return self.model.predict(x)
    
    def plot_hist(self, feature):
        plt.title(feature)
        x = np.arange(len(self.groups[feature]))
        y = [self.groups[feature][value] for value in self.groups[feature]]
        plt.bar(x, height=y)
        plt.xticks([])
        plt.ylabel("Probability")
        plt.xlabel("Data")
        plt.show()
        
    def count_updates(self):
        if len(self.features) <= 0:
            return 0
        return sum(self.features[list(self.features.keys())[0]])
        
        
        
    def plot_hists(self, name):
        fig, axs = plt.subplots(
            1, len(self.features), 
            sharey=True, tight_layout=True, 
            figsize=(8, 4)
        )
        
        for f, i in zip(self.features.keys(), range(len(self.features))):
            axs[i].title.set_text(f)
            x = np.arange(len(self.groups[f]))
            y = [self.groups[f][value] for value in self.groups[f]]
            axs[i].bar(x, height=y)
            axs[i].set_xticks([], minor=False)
            axs[i].set_ylabel("Probability")
            axs[i].set_xlabel("Data")
            s = sum(y)
            
        fig.suptitle(name + f', #records: {s}')
        plt.show()
        

In [9]:
#Worker 
class Worker:
    #secret_features must of the form {'f_name_1': 'f_value_1', 'f_name_2': 'f_value_2', ...}
    def __init__(self, id_w, zkp_client_prototype, server, x, y, secret_features, encrypt=True):
        self.id = id_w
        self.zkp_client_prototype = zkp_client_prototype
        self.server = server
        self.model = {}
        self.x = x
        self.y = y
        self.secret_features = secret_features
        self.encrypt = encrypt
        self.zkp_clients = {}
        for f in secret_features.keys():
            c = copy.copy(zkp_client_prototype)
            c.feature = f
            c.set_label(secret_features[f])
            self.zkp_clients[f] = c
        
    def send_registration(self):
        self.server.register_worker(self)
        
    def prepare_features(self):
        feature_to_send = []
        for k in self.zkp_clients.keys():
            c = self.zkp_clients[k]
            secret = c.encrypt_label(self.encrypt)
            feature_to_send += [{c.feature: secret}]
        return feature_to_send
                    
    def load_model(self, model):
        self.model = model
        
    def send_update(self, features, weights):
        self.server.load_update(self.id, features, weights)
        
    def train(self):
        self.model.train(self.x, self.y)
        features = self.prepare_features()
        weights = self.model.get_weights()
        self.send_update(features, weights)

In [10]:
class LearningModel:
    def __init__(self, model, epochs=epochs_n, learning_rate=learning_rate):
        self.model = model
        self.epochs = epochs
        self.learning_rate = learning_rate
    
    def set_weights(self, W):
        self.model.load_weights(W)
        
    def get_weights(self):
        return self.model.get_weights()
    
    def train(self, X, y):
        self.model.fit(X, y, self.epochs, self.learning_rate)
    
    def predict(self, x):
        return self.model.predict(x)

### ZKP-FED-Framework

In [11]:
class ZKPFEDFramework:
    def __init__(self, features, not_features, gap=0.05, max_gap=5, data=[], self_balancing=True):
        self.self_balancing = self_balancing
        self.client_prototype = ClientInitializer().generate_client_prototype()
        self.features = features
        self.not_features = not_features #TESTING
        self.server_initializer = ServerInitializer(features, self.client_prototype)
        self.gap = gap
        self.max_gap = max_gap

        #self.test_server()
        
        
    #ZKP Assertions
    def test_server(self):
        #Create a mock server initializer with actual features
        si_f = ServerInitializer(self.features, self.client_prototype)
        
        #Create a mock server initializer with unauthorized features (not_features)
        si_n = ServerInitializer(self.not_features, self.client_prototype)
        
        #Create a mock server
        server = ZKServer('password', features)
        
        #Register authrized clients in the mock server
        server.init_zk(si_f.create_clients())
        
        #Create proofs for authorized users
        mock_clients_f = si_f.create_clients()
        proofs_f = [server.verify(c.feature, c.create_proofs(server.tokens[c.feature])) 
                  for c 
                  in mock_clients_f]
        
        #Create proofs for unauthorized users
        mock_clients_n = si_n.create_clients()
        proofs_n = [server.verify_clients(c.feature, c.create_proof(server.tokens[c.feature][0])) 
                  for c 
                  in mock_clients_n]
        
        #Assert that all the registered clients get access to the server
        assert sum(proofs_f) == len(mock_clients_f)
        
        #Assert that none of the non-registered clients get access to the server
        assert sum(proofs_n) == 0
        
    #Create a server
    def create_server(self, model):
        fs = FairServer('password', features, model, gap=self.gap, max_gap=self.max_gap, self_balancing=self.self_balancing)
        fs.init_zk(self.server_initializer.create_clients())
        return fs
    
    #Create workers
    def create_workers(self, server, X, y, secret_features):
        return [Worker(i, self.client_prototype, server, X[i], y[i], secret_features[i]) for i in range(len(X))]

    #Register all workers
    def register_workers(self, workers):
        for worker in workers:
            worker.send_registration()

    def start_training(self, server):
        start_train = time.time()
        server.train()
        end_train = time.time()
        #print('Training time:' + str(end_train - start_train))

    def zkp_fed_training(self, model, X, y, secret):
        s = self.create_server(model)
        ws = self.create_workers(s, X, y, secret)
        self.register_workers(ws)
        self.start_training(s)
        #print(s.fair_metrics())
        return s#, ws

## Evaluation

### Model for reference

In [12]:
mult_fact = 2

In [13]:
# network (not federated)
net = nn.get_3l_nn(2, 2, 1)

X = np.array([[[[0, 0]]], [[[0, 1]]], [[[1, 0]]], [[[1, 1]]]] * mult_fact)
y = np.array([[[[0]]], [[[1]]], [[[1]]], [[[0]]]] * mult_fact)

#train
for i in range(len(X)):
    net.fit(X[i], y[i], epochs=epochs_n, learning_rate=learning_rate)

# test
out = net.predict(X)

### Model for Fed-Learning

In [14]:
X = np.array([[[[0, 0]]], [[[0, 1]]], [[[1, 0]]], [[[1, 1]]], [[[1, 2]]]] * mult_fact)
y = np.array([[[[0]]], [[[1]]], [[[1]]], [[[0]]], [[[2]]]] * mult_fact)

zkp_fed_framework = ZKPFEDFramework(features, not_features, gap=100, max_gap=mult_fact*100)
# network (for federated learning)
net2 = nn.get_3l_nn(2, 2, 1)

feature_groups = [
    {
        'ETHNICITY': get_rnd_ethnicity(),
        'GENDER': get_rnd_gender()
    } for _ in range(int(len(X)/mult_fact) - 1)] + [
    {
        'ETHNICITY': 'unauthorized',
        'GENDER': get_rnd_gender()
    }
]

feature_groups *= mult_fact

#train
#s = zkp_fed_framework.zkp_fed_training(net2, X, y, feature_groups)

#test
#out2 = s.predict([w.x for w in s.workers])

### Test if the 2 models behave in the same way

In [15]:
#assert np.array_equal(out, out2)

### Test if workers retain features

In [16]:
def test_features(z_fed):
    w_groups = [w.secret_features for w in z_fed.s.workers] 
    test_groups = {}

    salt = z_fed.client_prototype.zk.params.s

    for f in features:
        test_groups[f] = Counter()

    for t in w_groups:
        for f in t.keys():
            secret = hashlib.sha256()
            label = t[f] + str(salt)
            secret.update(label.encode('utf-8'))
            label = secret.hexdigest()
            test_groups[f].update({label : 1})

    #check if the two counters are equal
    assert test_groups == z_fed.s.groups

## Usage

### Read data

In [17]:
df = pd.read_csv('../dataset/data.csv')

### Clean data

In [18]:
#Remove other ethnicities
del p['ETHNICITIES'][4]
df = df[df['ethnicity'] != 4]
#print_summary(df)

In [19]:
def age_filter(df, age_min, age_max):
    return (df['age'] >= age_min) & (df['age'] <= age_max)

def ethnicity_gender_filter(df, i, j):
    return (df['ethnicity'] == p['ETHNICITIES'][i]) & (df['gender'] == p['GENDERS'][j])

def ethnicity_age_filter(df, e, age_min, age_max):
    return (df['ethnicity'] == p['ETHNICITIES'][e]) & age_filter(df, age_min, age_max)

def ethnicity_gender_age_filter(df, i, j, age_min, age_max):
    return ethnicity_gender_filter(df, i, j) & age_filter(df, age_min, age_max)

In [20]:
#Prepare
df = df.loc[age_filter(df, p['age_min'], p['age_max'])]
df['ethnicity'] = df['ethnicity'].map(p['ETHNICITIES'])
df['gender'] = df['gender'].map(p['GENDERS'])

### Create balanced dataset

In [21]:
#Ethnicity gender age sample size
ethnicity_age_sample_size = round(p['data_samples'] / (round(p['age_range']() / p['age_bins']) * len(p['ETHNICITIES'])))

balanced_df = pd.concat([
    df.loc[ethnicity_age_filter(df, e, age_m, age_M)]\
        .sample(ethnicity_age_sample_size, random_state=p['seed'], replace=True)
    for e in range(len(p['ETHNICITIES']))
    #for j in range(len(GENDERS))
    for (age_m, age_M) in p['bins']()
])

### Create imbalanced dataset

In [22]:
def create_unbalanced_set(df, minority_proportion=0.05, majority_group=0):
    min_factor = minority_proportion # 5% of the population 
    maj_factor = 1 - min_factor

    #Size of minority group
    min_sample_size = round(p['data_samples'] * min_factor / len(p['bins']()) / (len(p['ETHNICITIES']) - 1))

    #Size of majority group
    maj_sample_size = round(p['data_samples'] * maj_factor / len(p['GENDERS']) / len(p['bins']()))

    maj_male = pd.concat([
        df.loc[ethnicity_gender_age_filter(df, majority_group, 0, age_min, age_max)]\
            .sample(maj_sample_size, random_state=p['seed'], replace=True)
        for (age_min, age_max) in p['bins']()
    ])

    maj_female = pd.concat([
        df.loc[ethnicity_gender_age_filter(df, majority_group, 1, age_min, age_max)]\
            .sample(maj_sample_size, random_state=p['seed'], replace=True)
        for (age_min, age_max) in p['bins']()
    ])

    unbalanced_df = pd.concat([
        df.loc[ethnicity_age_filter(df, e, age_min, age_max)]\
            .sample(min_sample_size, random_state=p['seed'], replace=True)
        for e in range(len(p['ETHNICITIES']))
        #for j in range(len(GENDERS))
        for (age_min, age_max) in p['bins']()
        if e != majority_group
    ] + [maj_female, maj_male])
    
    return unbalanced_df

### Model

In [23]:
def standard_nn():
    return nn.get_standard_nn(p['seed'])

### Evaluate predictive models

In [24]:
def evaluate_ethnicity(dataset, ethnicity, df_test, model):
    idx_keys = dataset.index.intersection(df_test[df_test['ethnicity'] == ethnicity].index)
    ethn_df = dataset.loc[idx_keys]
    ethn_X = prepare_X(ethn_df)
    ethn_y = prepare_y(ethn_df)
    ethn_y_pred = model.predict(ethn_X)
    score = f1_score(convert_output(ethn_y), convert_output(ethn_y_pred), average='macro')
    return score

def evaluate_all_ethnicities(dataset, df_test, model):
    scores = [evaluate_ethnicity(dataset, e, df_test, model) for e in p['ETHNICITIES'].values()]
    return dict(zip(p['ETHNICITIES'].values(), scores))

def get_ethnicity_predictions(dataset, df_test, ethnicity, model):
    idx_keys = dataset.index.intersection(df_test[df_test['ethnicity'] == ethnicity].index)
    ethn_df = dataset.loc[idx_keys]
    ethn_X = prepare_X(ethn_df)
    ethn_y = prepare_y(ethn_df)         #y_true 
    ethn_y_pred = model.predict(ethn_X) #y_pred 
    return ethn_y, ethn_y_pred

def ethnicity_equality_metric(
    dataset, 
    df_test, 
    ethnicity, 
    model, 
    privileged_group, 
    ethn_y_p, 
    ethn_y_p_pred, 
    metric_fun):
    if ethnicity == privileged_group:
        return 0
    
    ethn_y_u, ethn_y_u_pred = get_ethnicity_predictions(dataset, df_test, ethnicity, model) #unprivileged
    
    #print(ethn_y_u, ethn_y_u, ethn_y_u_pred, ethnicity)
    
    return np.nanmean(metric_fun(
        one_hot_vect_to_class(convert_output(ethn_y_p)), 
        one_hot_vect_to_class(convert_output(ethn_y_p_pred)), 
        one_hot_vect_to_class(convert_output(ethn_y_u)), 
        one_hot_vect_to_class(convert_output(ethn_y_u_pred))
    ))
    

def all_ethnicity_equal_difference(
    dataset, df_test, model, privileged_group, ethn_y_p, ethn_y_p_pred, metric_fn_list
):
    scores = [
        np.nanmean([
            ethnicity_equality_metric(
                dataset, 
                df_test, 
                e, 
                model, 
                privileged_group, 
                ethn_y_p, 
                ethn_y_p_pred,
                metric_fn
            ) 
            for e in p['ETHNICITIES'].values()
        ]) for metric_fn in metric_fn_list
    ]
    metrics_list = [m.__name__.replace('_', ' ').capitalize() for m in metric_fn_list]
    return dict(zip(metrics_list, scores)) # + scores_eodd

### Predict and evaluate

In [25]:
def predict_and_evaluate(df, df_test, model, X_test, y_test, privileged_group, exp_id):
        
    y_pred = model.predict(X_test)
    ethn_y_p, ethn_y_p_pred = get_ethnicity_predictions(df, df_test, privileged_group, model) #privileged
    
    #print(ethn_y_p, ethn_y_p_pred)
    
    all_ethnicities_f1s = evaluate_all_ethnicities(df, df_test, model)
    
    ev = dict()
    ev['experiment ID'] = exp_id
    ev['data sample #'] = round(len(df))
    ev['f1'] = f1_score(convert_output(y_test), convert_output(y_pred), average='macro')
    ev['f1 variance'] = macro_variance(ev['f1'], list(all_ethnicities_f1s.values()))
    #ev.update(all_ethnicities_f1s)
    all_parities = all_ethnicity_equal_difference(
        df, df_test, model, privileged_group, ethn_y_p, ethn_y_p_pred,
        [
            disparate_impact_ratio, 
            statistical_parity_difference, 
            equal_opportunity_difference,
            equal_odd_difference
        ]
    )
    ev.update(all_parities)
    #ev['EOR'] = equal_opportunity_difference(convert_output(y_test), convert_output(y_pred))
    #print_summary(df, exp_id)
    
    return ev

def compare_model_behaviour(
    df_list, df_test_list, 
    model_list, 
    X_test_list, y_test_list, 
    privileged_group_list, 
    exp_ids
):
    return [
        predict_and_evaluate(
            df_list[i], df_test_list[i], 
            model_list[i], 
            X_test_list[i], y_test_list[i], 
            privileged_group_list[i], 
            exp_ids[i]
        ) 
        for i in range(len(df_list))
    ]

### Create pair of self-balanced - imbalanced Z-Fed Framework
The trained model will be extracted from the FairServer

In [26]:
def create_and_train_zfed(
    features,
    not_features,
    dataset,
    privileged_group, #numeric
    gap=0.05,
    max_gap=1,
    self_balancing=True,
    privileged_group_proportion=0.95     
):
    zkp_fed_framework = ZKPFEDFramework(
        features, not_features, 
        gap=gap, max_gap=max_gap, 
        self_balancing=self_balancing
    )
    unprivileged_proportion = 1 - privileged_group_proportion
    agument_fact = 1 if not self_balancing else round(1 / unprivileged_proportion)
    df = pd.concat([
        create_unbalanced_set(dataset, unprivileged_proportion, privileged_group)
    ] * agument_fact)        
    df_train, df_test = create_train_test(df)
    df_train = df_train.reset_index()
    df_test = df_test.reset_index()
    X_train, y_train, X_test, y_test = prepare_data(df_train, df_test)

    feature_groups = [
        {
            'ETHNICITY': df_train.iloc[i]['ethnicity'],
            'GENDER': df_train.iloc[i]['gender']
        } 
        for i in df_train.index
    ]

    #train
    s = zkp_fed_framework.zkp_fed_training(standard_nn(), X_train, y_train, feature_groups)
    return df, df_test, s, X_test, y_test

In [27]:
def train_compare_pair_balanced_umbalanced(
    features,
    not_features,
    dataset,
    privileged_group, #numeric
    gap=0.05,
    max_gap=1,
    privileged_group_proportion=0.95 
):
    df_b, df_test_b, s_b, X_test_b, y_test_b = create_and_train_zfed(
        features,
        not_features,
        dataset,
        privileged_group, #numeric
        gap=gap,
        max_gap=max_gap,
        self_balancing=True,
        privileged_group_proportion=privileged_group_proportion
    )
    
    df_u, df_test_u, s_u, X_test_u, y_test_u = create_and_train_zfed(
        features,
        not_features,
        dataset,
        privileged_group, #numeric
        gap=gap,
        max_gap=max_gap,
        self_balancing=False,
        privileged_group_proportion=privileged_group_proportion
    )

    metrics = compare_model_behaviour(
        [df_b, df_u], [df_test_b, df_test_u],
        [s_b.model, s_u.model],
        [X_test_b, X_test_u], [y_test_b, y_test_u],
        [p['ETHNICITIES'][privileged_group]] * 2,    
        ['Self balanced [agumented]', 'Imbalanced']
    )
    
    s_b.plot_hists('Self balanced [agumented]')
    s_u.plot_hists('Imbalanced')
    
    return get_metrics_data(metrics)

### Get results

In [ ]:
results = [
    train_compare_pair_balanced_umbalanced(
        features,
        not_features,
        df,
        i, #numeric
        gap=0.05,
        max_gap=1,
        privileged_group_proportion=0.9 
    )
    for i in range(len(p['ETHNICITIES']))
]

### Display results

In [ ]:
for r in results:
    display(r)
    print('')